In [1]:
import pandas as pd
import sys

sys.path.insert(0,'../satori')

from postprocess import *

In [3]:
exp_name = "../results/HP/E2/"
interaction_file = "Interactions_SATORI/interactions_summary_attnLimit-0.019129996827528525.txt"
df,suffix = pd.read_csv(exp_name + interaction_file ,sep='\t'),'SATORI'
output_path = exp_name
df = df[df['adjusted_pval'] < 0.05]
df.shape


(721, 12)

In [7]:
hp_cluster_info = "/s/chromatin/a/nobackup/Saira/SATORI_ENHANCEMENTS/TIANA_demo_upload/mycluster/hp//motifs_summary.csv"
hp_cluster = pd.read_csv(hp_cluster_info, delimiter=',')

In [8]:
hp_cluster

,Clustered_Motif,Contributing_Motifs
0,MAFB,>M6331_1.02
1,TLX2,>M0891_1.02
2,PPARA_1_merged,>PPARA|RARA|RARB
3,ZFY,>M1963_1.02
4,IRX1,>M6037_1.02
...,...,...
500,ETS2,>M6221_1.02
501,HOXB7_1_merged,>HOXB7|HOXB8|HOXA5
502,LHX1_1_merged,>LHX1|LHX5
503,ELF1_1_merged,>ELF1|FLI1


In [24]:
# Remove '>' and split motifs by '|', then map to cluster name
def map_index_to_motifs(df):
    index_to_cluster = {}
    for idx, row in df.iterrows():
        motifs_str = row['Contributing_Motifs'].lstrip('>')
        if '|' in motifs_str:
            motifs = motifs_str.split('|')
            index_to_cluster[idx] =  [x.upper() if isinstance(x, str) else x for x in motifs]
        else:
            index_to_cluster[idx] = row['Clustered_Motif']
    return index_to_cluster



In [ ]:
index_to_cluster_map = map_index_to_motifs(hp_cluster)

{0: 'MAFB',
 1: 'TLX2',
 2: ['PPARA', 'RARA', 'RARB'],
 3: 'ZFY',
 4: 'IRX1',
 5: 'MECOM',
 6: 'ZSCAN10',
 7: 'ATF2',
 8: ['PAX3', 'PAX7'],
 9: 'SOX17',
 10: 'FOXA2',
 11: 'DMRT3',
 12: ['POU5F1', 'SOX2'],
 13: 'LBX1',
 14: 'SPZ1',
 15: 'ZBTB12',
 16: ['SOX3', 'SOX12', 'SOX11'],
 17: ['FOXK1', 'FOXK2', 'FOXO4'],
 18: 'NR3C1',
 19: 'PDX1',
 20: ['GLIS3', 'ZIC5'],
 21: ['LMX1A', 'LMX1B', 'VTN', 'VENTX'],
 22: 'CDX2',
 23: 'MECP2',
 24: 'E2F2',
 25: ['MEIS2', 'PKNOX2', 'TGIF2', 'TGIF2LX'],
 26: 'MBD2',
 27: 'CUX2',
 28: ['ESRRG', 'ESRRB'],
 29: 'ZBTB7B',
 30: 'SPI1',
 31: 'ENSG00000250542',
 32: 'NR0B1',
 33: 'HHEX',
 34: ['MSX1', 'BARX1'],
 35: 'NR6A1',
 36: 'ZNF691',
 37: 'TLX1',
 38: ['OLIG1',
  'OLIG2',
  'OLIG3',
  'NEUROG2',
  'NEUROG1',
  'NEUROD2',
  'BHLHA15',
  'BHLHE22',
  'BHLHE23'],
 39: 'MEIS1',
 40: 'ZNF354C',
 41: ['ETV2', 'ERG'],
 42: ['HSF2', 'HSF4'],
 43: 'NFKB2',
 44: 'HOXD10',
 45: 'ZFHX3',
 46: ['SREBF1', 'SREBF2'],
 47: ['USF1', 'ATF3'],
 48: 'DNMT1',
 49: ['MXI1', 

In [3]:
def get_interaction_df(df, index_to_cluster_map):
    
    import re
    # Extract filter indices from 'filter_interaction'
    def extract_filters(s):
        return list(map(int, re.findall(r'filter(\d+)', s)))

    # Build rows
    rows = []
    for _, row in df.iterrows():
        filters = extract_filters(row['filter_interaction'])
        if len(filters) == 2:
            filter_a, filter_b = filters
            motif_a = index_to_cluster_map.get(filter_a, f"filter{filter_a}")
            motif_b = index_to_cluster_map.get(filter_b, f"filter{filter_b}")
            
            rows.append({
                'filter_a': filter_a,
                'filter_b': filter_b,
                'motif_a_name': motif_a,
                'motif_b_name': motif_b,
                'adjusted_pval': row['adjusted_pval']  # assumes this column exists
            })

    # Create final DataFrame
    result_df = pd.DataFrame(rows)
    
    return result_df

In [ ]:
result_df = get_interaction_df(df)

,filter_a,filter_b,motif_a_name,motif_b_name,adjusted_pval
0,0,235,MAFB,TFCP2L1,7.713987e-03
1,0,249,MAFB,PPARG,1.630869e-02
2,0,265,MAFB,ZNF143,7.713987e-03
3,0,340,MAFB,"[ETS1, SMARCC2]",1.630869e-02
4,0,362,MAFB,REST,3.699511e-03
...,...,...,...,...,...
716,428,498,KLF3,NRF1,3.699511e-03
717,429,435,"[SP8, KLF14, KLF16]",CTCF,7.091305e-07
718,434,435,PAX8,CTCF,8.460580e-04
719,435,473,CTCF,TFAP2D,9.679096e-05


In [ ]:
df = pd.read_csv("/s/chromatin/p/nobackup/Saira/motif_databases/Human_Chromatin/hippie/hippie_current.txt", delimiter="\t", header=None)
df[0] = df[0].str.split('_').str[0]
df[2] = df[2].str.split('_').str[0]
tfs = set()
for i, row in df.iterrows():
    
    tfs.add((row[0] , row[2]))
    tfs.add((row[2] , row[0]))

In [32]:
def find_matches_in_db(result_df, tfs):
    matches_in_db = []
    for i, row in result_df.iterrows():
        if type(row['motif_a_name']) == str and type(row['motif_b_name']) == str:
            if (row['motif_a_name'].upper(), row['motif_b_name'].upper()) in tfs:
                matches_in_db.append(row)
        elif type(row['motif_b_name']) == str and type(row['motif_a_name']) == list:
            for motif in row['motif_a_name']:
                if (motif.upper(), row['motif_b_name'].upper()) in tfs:
                    matches_in_db.append(row)
        elif type(row['motif_a_name']) == str and type(row['motif_b_name']) == list:
            for motif in row['motif_b_name']:
                if (row['motif_a_name'].upper(), motif.upper()) in tfs:
                    matches_in_db.append(row)
        elif type(row['motif_a_name']) == list and type(row['motif_b_name']) == list:
            for motif_a in row['motif_a_name']:
                for motif_b in row['motif_b_name']:
                    if (motif_a.upper(), motif_b.upper()) in tfs:
                        matches_in_db.append(row)
    return matches_in_db



In [ ]:
matches_in_db = find_matches_in_db(result_df)
# Convert to DataFrame
matches_in_db = pd.DataFrame(matches_in_db)
matches_in_db.reset_index(drop=True, inplace=True)
matches_in_db.drop_duplicates(subset=['filter_a', 'filter_b'], inplace=True)
matches_in_db.reset_index(drop=True, inplace=True)
matches_in_db

,filter_a,filter_b,motif_a_name,motif_b_name,adjusted_pval
0,26,237,MBD2,ESR1,7.713987e-03
1,62,472,EPAS1,USF2,3.452692e-02
2,104,236,"[SP2, SP1]",NFYA,8.399691e-04
3,104,249,"[SP2, SP1]",PPARG,4.166337e-04
4,108,236,ZHX1,NFYA,4.166337e-04
5,112,130,MAZ,JUN,8.460580e-04
6,130,363,JUN,PURA,1.630869e-02
7,167,172,TGIF1,SMAD1,3.452692e-02
8,172,183,SMAD1,"[GLI3, GLI1, GLI2]",3.452692e-02
9,173,288,TFCP2,TFAP4,1.096421e-05


In [ ]:
# Load your file (assuming it's a whitespace-separated plain text file)
motifs_table = pd.read_csv(exp_name + "Motif_Analysis/table.txt", delim_whitespace=True)

# Filter for rows where ic > 3
filtered_indices = motifs_table[motifs_table['ic'] > 3].index.tolist()
len(filtered_indices)
motif_names = [index_to_cluster_map[i] for i in filtered_indices if i in index_to_cluster_map]
all_combinations = len(motif_names) * (len(motif_names) - 1) / 2
print("Number of combinations: ", all_combinations)


463

In [20]:
def get_hits_for_all_motfs(motif_names, tfs):
    success_in_population = 0
    for i in range(len(motif_names)):
        for j in range(i + 1, len(motif_names)):
            motif_a = motif_names[i]
            motif_b = motif_names[j]
            flag = False
            if type(motif_a) == list and type(motif_b) == list:
                for a in motif_a:
                    for b in motif_b:
                        if (a, b) in tfs or (b, a) in tfs:
                            flag = True
                            break
                if flag:
                    success_in_population += 1
                            
            elif type(motif_a) == list and type(motif_b) == str:
                for a in motif_a:
                    if (a, motif_b) in tfs or (motif_b, a) in tfs:
                        flag = True
                        break
                if flag:
                    success_in_population += 1
            elif type(motif_a) == str and type(motif_b) == list:
                for b in motif_b:
                    if (motif_a, b) in tfs or (b, motif_a) in tfs:
                        flag = True
                        break
                if flag:
                    success_in_population += 1
                    
            elif type(motif_a) == str and type(motif_b) == str:
                if motif_a != motif_b:
                    if (motif_a, motif_b) in tfs or (motif_b, motif_a) in tfs:
                        flag = True
                        success_in_population += 1
    print("Success in population: ", success_in_population)
    return success_in_population
success_in_population = get_hits_for_all_motfs(motif_names, tfs)


Success in population:  223


In [36]:
import scipy.stats as stats
from scipy.stats import hypergeom
# Calculate the p-value using hypergeometric distribution
N = all_combinations
K = success_in_population
n = df.shape[0]
k = matches_in_db.shape[0]
pval = hypergeom.sf(K-1, N, n, k)
print("P-value: ", pval)

P-value:  0.0


In [39]:
n,k, N, K , K/N , k/n

(721, 19, 106953.0, 2469, 0.023084906454236907, 0.026352288488210817)

In [6]:
exp_name = "../results/Arabidopsis/PSSM_E2/"
interaction_file = "Interactions_SATORI/interactions_summary_attnLimit-0.016960513483247723.txt"
df,suffix = pd.read_csv(exp_name + interaction_file ,sep='\t'),'SATORI'
output_path = exp_name
df = df[df['adjusted_pval'] < 0.05]
df.shape

(233, 12)

In [22]:
arab_cluster_info = "/s/chromatin/a/nobackup/Saira/SATORI_ENHANCEMENTS/TIANA_demo_upload/mycluster/arab//motifs_summary.csv"
arab_cluster = pd.read_csv(arab_cluster_info, delimiter=',')

In [25]:
index_to_cluster_map = map_index_to_motifs(arab_cluster)

In [26]:
# Load your file (assuming it's a whitespace-separated plain text file)
motifs_table = pd.read_csv(exp_name + "Motif_Analysis/table.txt", delim_whitespace=True)

# Filter for rows where ic > 3
filtered_indices = motifs_table[motifs_table['ic'] > 3].index.tolist()
len(filtered_indices)
motif_names = [index_to_cluster_map[i] for i in filtered_indices if i in index_to_cluster_map]
all_combinations = len(motif_names) * (len(motif_names) - 1) / 2
print("Number of combinations: ", all_combinations)


Number of combinations:  58311.0


In [29]:
arab_df = get_interaction_df(df, index_to_cluster_map)

In [30]:
arab_df.head()

,filter_a,filter_b,motif_a_name,motif_b_name,adjusted_pval
0,1,26,ANAC034,"[AT4G18890, AT4G18890, AT4G36780, BZR1, BAM8, ...",0.042848
1,2,12,FAR1,AT1G63040,0.000193
2,2,15,FAR1,TZF9,0.023368
3,2,31,FAR1,bZIP50,0.042848
4,2,35,FAR1,AT2G38300,0.023368


In [27]:
df_intrs = pd.read_csv("/s/chromatin/p/nobackup/Saira/motif_databases/Arabidopsis/arath_interactions.csv", delimiter=",")

tfs = set()
for i, row in df_intrs.iterrows():
    tf1, tf2 = row[0], row[1]
    if pd.notna(tf1) and pd.notna(tf2):  # safer than np.isnan for mixed types
        tfs.add((tf1.upper(), tf2.upper()))
        tfs.add((tf2.upper(), tf1.upper()))

In [28]:
success_in_population = get_hits_for_all_motfs(motif_names, tfs)

Success in population:  318


In [33]:
matches_in_db = find_matches_in_db(arab_df, tfs)
matches_in_db

[]

In [ ]:


# Convert to DataFrame
matches_in_db = pd.DataFrame(matches_in_db)
matches_in_db.reset_index(drop=True, inplace=True)
matches_in_db.drop_duplicates(subset=['filter_a', 'filter_b'], inplace=True)
matches_in_db.reset_index(drop=True, inplace=True)
matches_in_db

""


In [15]:
matches_in_db.shape

(0, 0)